# Install library requirements

In [1]:
# Install
!pip install tfx
!pip install opencv-python
!pip install -q kaggle
!sudo install docker

install: missing destination file operand after 'docker'
Try 'install --help' for more information.




---



---



---



# WRITE FILE: COMPONENTS.PY

In [2]:
COMPONENTS = "spam_components.py"

In [3]:
%%writefile {COMPONENTS}
"""
spam_components.py
"""
import os

import tensorflow_model_analysis as tfma
from tfx.components import (CsvExampleGen, Evaluator, ExampleValidator, Pusher,
                            SchemaGen, StatisticsGen, Trainer, Transform,
                            Tuner)
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import \
    LatestBlessedModelStrategy
from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


def init_components(
    data_dir,
    transform_module,
    trainer_module,
    tuner_module,
    train_steps,
    eval_steps,
    serving_model_dir,
):
    """Initiate tfx pipeline components

    Args:
        args (dict): args that containts some dependencies

    Returns:
        tuple: TFX pipeline components
    """
    output = example_gen_pb2.Output(
        split_config=example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ])
    )

    example_gen = CsvExampleGen(
        input_base=data_dir,
        output_config=output,
    )

    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"],
    )

    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"],
    )

    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs["statistics"],
        schema=schema_gen.outputs["schema"],
    )

    transform = Transform(
        examples=example_gen.outputs["examples"],
        schema=schema_gen.outputs["schema"],
        module_file=os.path.abspath(transform_module),
    )

    tuner = Tuner(
        module_file=os.path.abspath(tuner_module),
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        train_args=trainer_pb2.TrainArgs(
            splits=["train"],
            num_steps=train_steps,
        ),
        eval_args=trainer_pb2.EvalArgs(
            splits=["eval"],
            num_steps=eval_steps,
        ),
    )

    trainer = Trainer(
        module_file=trainer_module,
        examples=transform.outputs["transformed_examples"],
        transform_graph=transform.outputs["transform_graph"],
        schema=schema_gen.outputs["schema"],
        hyperparameters=tuner.outputs["best_hyperparameters"],
        train_args=trainer_pb2.TrainArgs(
            splits=["train"],
            num_steps=train_steps,
        ),
        eval_args=trainer_pb2.EvalArgs(
            splits=["eval"],
            num_steps=eval_steps
        ),
    )

    model_resolver = Resolver(
        strategy_class=LatestBlessedModelStrategy,
        model=Channel(type=Model),
        model_blessing=Channel(type=ModelBlessing),
    ).with_id("Latest_blessed_model_resolve")

    eval_config = tfma.EvalConfig(
        model_specs=[tfma.ModelSpec(label_key="label")],
        slicing_specs=[
            tfma.SlicingSpec(),
            tfma.SlicingSpec(feature_keys=["text"]),
        ],
        metrics_specs=[
            tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": .6},
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 1e-4},
                        ),
                    ),
                ),
            ]),
        ],
    )

    evaluator = Evaluator(
        examples=example_gen.outputs["examples"],
        model=trainer.outputs["model"],
        baseline_model=model_resolver.outputs["model"],
        eval_config=eval_config,
    )

    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=serving_model_dir,
            ),
        ),
    )

    return (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        tuner,
        trainer,
        model_resolver,
        evaluator,
        pusher,
    )

Writing spam_components.py


# WRITE FILE: PIPELINE.PY

In [4]:
PIPELINE = "spam_pipeline.py"

In [28]:
%%writefile {PIPELINE}
"""
spam_pipeline.py
"""
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline


def init_pipeline(pipeline_root: Text, pipeline_name, metadata_path, components):
    """Initiate tfx pipeline

    Args:
        pipeline_root (Text): a path to th pipeline directory
        pipeline_name (str): pipeline name
        metadata_path (str): a path to the metadata directory
        components (dict): tfx components

    Returns:
        pipeline.Pipeline: pipeline orchestration
    """

    logging.info(f"Pipeline root set to: {pipeline_root}")

    beam_args = [
        "--direct_running_mode=multi_processing",
        "----direct_num_workers=0",
    ]

    return pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path,
        ),
        eam_pipeline_args=beam_args,
    )

Writing spam_pipeline.py


# WRITE FILE: TRANSFORM.PY
Processing yang dilakukan hanya untuk merubah nama feature dan label yang telah di transform menjadi label_xf, text_xf, dan transform feature kedalam format lowercase string dan untuk casting label kedalam format int64 untuk memastikan tipe data pada label

In [7]:
TRANSFORM_MODULE_FILE = "spam_transform.py"

In [8]:
%%writefile {TRANSFORM_MODULE_FILE}
"""
spam_transform.py
"""
import tensorflow as tf
LABEL_KEY = "label"
FEATURE_KEY = "text"
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features

    Args:
        inputs: map from feature keys to raw features.

    Return:
        outputs: map from feature keys to transformed features.
    """

    outputs = {}

    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])

    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Writing spam_transform.py


# WRITE FILE: TUNER.PY
Melakukan tuning hyperparameter pada arsitektur model yang digunakan untuk mendapatkan hyperparameter terbaik.

In [9]:
TUNER_FILE = "tuner_module_file.py"

In [10]:
%%writefile {TUNER_FILE}
"""
tuner_module_file.py
"""
from typing import NamedTuple, Dict, Text, Any
import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from keras_tuner.engine import base_tuner
from tfx.components.trainer.fn_args_utils import FnArgs


LABEL_KEY = "label"
FEATURE_KEY = "text"
NUM_EPOCHS = 1

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
)


def transformed_name(key):
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=8):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )

    return dataset


def model_builder(hp, vectorizer_layer):
    num_hidden_layers = hp.Choice(
        "num_hidden_layers", values=[1, 2]
    )
    embed_dims = hp.Int(
        "embed_dims", min_value=16, max_value=128, step=32
    )
    lstm_units= hp.Int(
        "lstm_units", min_value=32, max_value=128, step=32
    )
    dense_units = hp.Int(
        "dense_units", min_value=32, max_value=256, step=32
    )
    dropout_rate = hp.Float(
        "dropout_rate", min_value=0.1, max_value=0.5, step=0.1
    )
    learning_rate = hp.Choice(
        "learning_rate", values=[1e-2, 1e-3, 1e-4]
    )

    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )

    x = vectorizer_layer(inputs)
    x = tf.keras.layers.Embedding(input_dim=5000, output_dim=embed_dims)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units))(x)

    for _ in range(num_hidden_layers):
        x = tf.keras.layers.Dense(dense_units, activation=tf.nn.relu)(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=["binary_accuracy"],
    )

    return model


def tuner_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(
        fn_args.train_files[0], tf_transform_output, NUM_EPOCHS
    )
    eval_set = input_fn(
        fn_args.eval_files[0], tf_transform_output, NUM_EPOCHS
    )

    vectorizer_dataset = train_set.map(
        lambda f, l: f[transformed_name(FEATURE_KEY)]
    )

    vectorizer_layer = tf.keras.layers.TextVectorization(
        max_tokens=5000,
        output_mode="int",
        output_sequence_length=500,
    )
    vectorizer_layer.adapt(vectorizer_dataset)

    tuner = kt.Hyperband(
        hypermodel=lambda hp: model_builder(hp, vectorizer_layer),
        objective=kt.Objective('binary_accuracy', direction='max'),
        max_epochs=NUM_EPOCHS,
        factor=3,
        directory=fn_args.working_dir,
        project_name="kt_hyperband",
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping_callback],
            "x": train_set,
            "validation_data": eval_set,
        },
    )

Writing tuner_module_file.py


# WRITE FILE: TRAINER.PY
Melatih model dengan menggunakan hyperparameter yang telah dituning sebelumnya

In [11]:
TRAINER = "spam_trainer.py"

In [12]:
%%writefile {TRAINER}
"""
spam_trainer.py
"""
import os
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "label"
FEATURE_KEY = "text"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset

# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = tf.keras.layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


EMBEDDING_DIM=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, name="embedding")(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)


    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]

    )

    # print(model)
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)


    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])

    # Build the model
    model = model_builder()


    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing spam_trainer.py


# WRITE FILE: DOCKERFILE

In [13]:
DOCKER = "Dockerfile"

In [14]:
%%writefile {DOCKER}
FROM tensorflow/serving:latest

COPY ./serving_model_dir /models
ENV MODEL_NAME=spam-detection-model

Writing Dockerfile


# WRITE FILE: REQUIREMENTS

In [15]:
REQ = "requirements.txt"

In [16]:
%%writefile {REQ}
# Install
!pip install tfx
!pip install opencv-python
!pip install -q kaggle
!sudo install docker

# Import dependencies
import pandas as pd
import numpy as np
import cv2
import os
import zipfile
import shutil
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import files
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import Sequential, load_model, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Activation, Add, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dense, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D, LSTM, Flatten
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_hub as hub
from tfx.components import Tuner
from tfx.proto import trainer_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.proto import pusher_pb2
import tensorflow_transform as tft

# Conda:
pip install pandas numpy opencv-python tensorflow matplotlib scikit-learn nltk tensorflow-hub tensorflow-model-analysis tensorflow-transform
pip install tfx

Writing requirements.txt


In [17]:
REQTF = "requirements-tfserving.txt"

In [18]:
%%writefile {REQTF}
tensorflow-serving-api
transformers

Writing requirements-tfserving.txt




---



---



---



# IMPORT LIBRARY

In [20]:
import os
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules.spam_pipeline import init_local_pipeline
from modules.spam_components import init_components

# SET VARIABEL

In [21]:
PIPELINE_NAME = 'kianaa19-pipeline'

DATA_ROOT = 'data'
TRANSFORM_MODULE_FILE = 'modules/spam_transform.py'
TUNER_MODULE_FILE = 'modules/tuner_module_file.py'
TRAINER_MODULE_FILE = 'modules/spam_trainer.py'

OUTPUT_BASE = 'output'
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, 'metadata.sqlite')

# RUN PIPELINE

In [ ]:
from modules.spam_pipeline1 import init_pipeline
from modules.spam_components import init_components

components = init_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    train_steps=1000,
    eval_steps=800,
    serving_model_dir=serving_model_dir,
)

pipeline = init_pipeline(
    pipeline_root,
    PIPELINE_NAME,
    metadata_path,
    components
)
BeamDagRunner().run(pipeline)

Search space summary
Default search space size: 6
num_hidden_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2], 'ordered': True}
embed_dims (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
lstm_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_hidden_layers
48                |48                |embed_dims
128               |128               |lstm_units
224               



---



---



---



# PYLINT

In [ ]:
!pip install pylint
!pip install autopep8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.6/515.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 845.7 kB/s eta 0:00:00


In [ ]:
!pylint spam_components.py

************* Module spam_components
spam_components.py:100:0: R1707: Disallow trailing comma tuple (trailing-comma-tuple)
spam_components.py:28:0: R0913: Too many arguments (6/5) (too-many-arguments)
spam_components.py:28:0: R0914: Too many local variables (20/15) (too-many-locals)

------------------------------------------------------------------
Your code has been rated at 8.85/10 (previous run: 8.15/10, +0.70)



In [ ]:
!autopep8 --in-place --aggressive --aggressive spam_components.py

In [ ]:
!pylint spam_components.py


-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 8.85/10, +1.15)



In [ ]:
!pylint spam_pipeline.py

************* Module spam_pipeline
spam_pipeline.py:24:4: W0621: Redefining name 'components' from outer scope (line 60) (redefined-outer-name)
spam_pipeline.py:24:16: W0621: Redefining name 'pipeline_root' from outer scope (line 19) (redefined-outer-name)

------------------------------------------------------------------
Your code has been rated at 9.13/10 (previous run: 9.13/10, +0.00)



In [ ]:
!autopep8 --in-place --aggressive --aggressive spam_pipeline.py

In [ ]:
!pylint spam_pipeline.py


-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 9.13/10, +0.87)



In [ ]:
!pylint spam_transform.py

************* Module spam_transform
spam_transform.py:1:0: C0114: Missing module docstring (missing-module-docstring)

-----------------------------------
Your code has been rated at 9.00/10



In [ ]:
!autopep8 --in-place --aggressive --aggressive spam_transform.py

In [ ]:
!pylint spam_transform.py


-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 9.00/10, +1.00)



In [ ]:
!pylint tuner_module_file.py

************* Module tuner_module_file
tuner_module_file.py:29:0: C0116: Missing function or method docstring (missing-function-docstring)
tuner_module_file.py:33:0: C0116: Missing function or method docstring (missing-function-docstring)
tuner_module_file.py:37:0: C0116: Missing function or method docstring (missing-function-docstring)
tuner_module_file.py:54:0: C0116: Missing function or method docstring (missing-function-docstring)
tuner_module_file.py:99:0: C0116: Missing function or method docstring (missing-function-docstring)

------------------------------------------------------------------
Your code has been rated at 8.91/10 (previous run: 5.53/10, +3.38)



In [ ]:
!autopep8 --in-place --aggressive --aggressive tuner_module_file.py

In [ ]:
!pylint tuner_module_file.py


-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 8.91/10, +1.09)



In [ ]:
!pylint spam_trainer.py

************* Module spam_trainer
spam_trainer.py:112:0: C0301: Line too long (108/100) (line-too-long)
spam_trainer.py:113:0: C0301: Line too long (145/100) (line-too-long)
spam_trainer.py:6:0: E0401: Unable to import 'tensorflow.keras' (import-error)
spam_trainer.py:57:0: C0103: Constant name "embedding_dim" doesn't conform to UPPER_CASE naming style (invalid-name)
spam_trainer.py:104:0: C0116: Missing function or method docstring (missing-function-docstring)
spam_trainer.py:7:0: C0411: standard import "os" should be placed before third party imports "tensorflow", "tensorflow_transform", "tensorflow.keras.layers" (wrong-import-order)
spam_trainer.py:8:0: W0611: Unused tensorflow_hub imported as hub (unused-import)

-----------------------------------
Your code has been rated at 8.00/10



In [ ]:
!autopep8 --in-place --aggressive --aggressive spam_trainer.py

In [ ]:
!pylint spam_trainer.py


-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 8.00/10, +2.00)

